In [1]:
%config IPCompleter.greedy=True

In [14]:
from influxdb import InfluxDBClient
import getpass
import pandas as pd

In [17]:
dataLocation = "../data/"
!export dataLocation="../data/"

In [79]:
name = input()
password = getpass.getpass()
client = InfluxDBClient('localhost', 8086, name, password, 'SG')

nimac
········


In [80]:
def makeJson(time, temp, humid, M1, M2, M3):
    return {
            "measurement": "garden",
            "tags": {
                "host": "server01",
                "region": "us-west"
            },
    #         "time": "2009-11-10T23:00:00Z",
            "time": str(time),
            "fields": {
                "temp": temp,
                "humid": humid,
                "M1": M1,
                "M2": M2,
                "M3": M3
            }
        }

In [84]:
def readCsv(dataLocation, csvFileName):
    df=pd.read_csv(dataLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

df = readCsv(dataLocation, "master")
df["time"] = pd.to_datetime(df['time'], unit='ms')
json_body = []

for i in range(len(df)):
    time, temp, humid, M1, M2, M3 = df.iloc[i]
    json_body.append(makeJson(time, temp, humid, M1, M2, M3))

In [85]:
client.write_points(json_body)

True

In [ ]:
result = client.query('select temp from garden;')
print("Result: {0}".format(result))